<a href="https://colab.research.google.com/github/R0b0t-Maker/LLM-T/blob/main/chatbot_LangChain_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pinecone**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -r "/content/drive/MyDrive/Udemy/LangChain/p-1/requirements.txt" -q
!pip install --quiet openai python-dotenv
!pip install -q pinecone-client


In [ ]:
!pip install --upgrade -q pinecone-client

In [ ]:
!pip show pinecone-client

Name: pinecone-client
Version: 4.1.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv("/content/drive/MyDrive/Udemy/LangChain/p-1/.env", override=True)

True

In [ ]:
from pinecone import Pinecone

pc=Pinecone()
# pc=Pinecone(api_key='e4179379-10db-4ee7-9082-a01a78cda928')
pc.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'testfile-apju9rk.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'testfile',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [ ]:
# pc.list_indexes()[0]

In [ ]:
# pc.list_indexes()[0]['name']

In [ ]:
from pinecone import ServerlessSpec

index_name= 'langchain'
if index_name not in pc.list_indexes().names():
  print(f'Creating index:{index_name}')
  pc.create_index(
      name= index_name,
      dimension=1536,
      metric='cosine',
      spec=ServerlessSpec(
          cloud = 'aws',
          region='us-east-1'
      )

  )
  print('Index created!')
else:
  print(f'Index{index_name} already exsits!')

Creating index:langchain
Index created!


In [ ]:
index_name='quickstart'
if index_name in pc.list_indexes().names():
  print(f'Deleting index{index_name}...')
  pc.delete_index(index_name)
  print('done!!!')
else:
  print(f'Index {index_name} does not exist!')

Index quickstart does not exist!


In [ ]:
index_name= 'langchain'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
#inserting vectors
import random

vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')

index_nmae = 'langchain'
index = pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [ ]:
#updating vectors
index.upsert(vectors=[('c', [0.5]*1536)])
#fetching vectors

index.fetch(ids=['c','d'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [ ]:
# #deleting vectors
# index.delete(ids=['b', 'a'])

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [ ]:
#query
query_vector = [random.random() for _ in range(1536)]
index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [], 'namespace': '', 'usage': {'read_units': 1}}

**Namespace**

In [ ]:
# index.describe_index_stats()

import random

vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [ ]:
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
# print(vectors)
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors), namespace='first-namespace')

{'upserted_count': 3}

In [ ]:
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
# print(vectors)
ids = list('qp')
index.upsert(vectors=zip(ids, vectors), namespace='second-namespace')

{'upserted_count': 2}

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [ ]:
index.fetch(ids=['x'], namespace='first-namespace')

{'namespace': 'first-namespace', 'usage': {'read_units': 1}, 'vectors': {}}

In [ ]:
index.delete(ids=['x'], namespace='first-namespace')

{}

In [ ]:
index.delete(delete_all=True, namespace='first-namespace')

{}

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

**Splitting and Embedding Text Using Langhain**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv("/content/drive/MyDrive/Udemy/LangChain/p-1/.env", override=True)

True

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
with open ('/content/drive/MyDrive/Udemy/LangChain/p-1/robot-history.txt') as f:
  testfile = f.read()

text_splitter= RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap= 20,
    length_function=len
)

In [ ]:
chunks=text_splitter.create_documents([testfile])
# print(chunks[1])
print(chunks[1].page_content)

bird he called "The Pigeon", which was propelled by steam.[13] Taking up the earlier reference in


In [ ]:
print(f'Number of chunks is: {len(chunks)}')

Number of chunks is: 121


**Embedding Cost**

In [ ]:
!pip install --upgrade tiktoken


In [ ]:
def print_embedding_cost(texts):
    import tiktoken

    # Explicitly get the encoding using get_encoding
    enc = tiktoken.get_encoding('cl100k_base')

    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')

print_embedding_cost(chunks)


Total Tokens: 2440
Embedding Cost in USD: 0.000976


In [ ]:
!pip install --upgrade openai langchain


  Using cached openai-1.30.2-py3-none-any.whl (320 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()
vector = embedding.embed_query(chunks[0].page_content)
print(vector)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[-0.008847972054759063, -0.005676560028270197, -0.008021805986599866, -0.023932164316667715, 0.021826773518991475, -0.001703134622101106, -0.045545733869293455, -0.024718355223209986, 0.009181103368620852, -0.020827378646083472, 0.025930954062483726, 0.006709267846299854, -0.0076953368892333516, -0.02687704747813898, 0.003424591387302186, 0.0027133558878522576, -0.001151801981010148, 0.051035739262840336, 0.011559662272072172, 0.011386433467323364, -0.025571171088672923, 0.007515445867989269, -0.0015215779303178747, -0.029928531691470472, 0.003244700366058103, 0.009167778004307665, 0.017136282533654776, -0.04149485664569924, 0.007262265808683971, -0.012965477031241863, 0.03363295130556707, -0.019028469364965283, -0.029582074081972855, -0.018109026677936405, -0.049276806074661714, -0.016350091627112505, 0.006222895774159039, -0.005539976441043976, 0.01986795986611503, 0.013611751612495658, 0.01591035833006785, 0.008701393978636632, 0.01048697882676427, -0.0006287856226693852, -0.0076087

 **Inserting the Embeddings into a Pinecone Index**

In [ ]:
import pinecone
from langchain.vectorstores import Pinecone
pc = pinecone.Pinecone()

In [ ]:
for i in pc.list_indexes().names():
  print('Deleting all indexes ...', end='')
  pc.delete_index(i)
  print('Done')

Deleting all indexes ...Done
Deleting all indexes ...Done


In [ ]:
index_name = 'testfile'
if index_name not in pc.list_indexes().names():
  print(f'Creating index {index_name} ...')
  pc.create_index(
      name=index_name,
      dimension=1536,
      metric='cosine',
      spec=pinecone.PodSpec(
          environment='gcp-starter'
      )
  )
  print('done..!!!')

Creating index testfile ...
done..!!!


In [ ]:
vector_store= Pinecone.from_documents(chunks, embedding, index_name= index_name)

In [ ]:
#loading the vector store from an existing index
vector_store = Pinecone.from_existing_index(index_name='testfile', embedding=embedding)

**Asking Questions (Similarity Search)**

In [ ]:
query = 'what is the first signs of robots?'
result = vector_store.similarity_search(query)
print(result)

[]


In [ ]:
for r in result:
  print(r.page_content)
  print('-'*50)

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model= 'gpt-3.5-turbo', temperature = 1)
retriever = vector_store. as_retriever(SearchType= 'similarity', search_kwargs= {'k':3})
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever= retriever)

In [ ]:
query = 'what is the first signs of robots?'
data = chain.invoke(query)
print(f"Query: {data['query']}\nResult: {data['result']}")


Query: what is the first signs of robots?
Result: I'm not sure what you mean by "the first signs of robots." Are you asking about the initial development of robots or how robots are identified or detected? Can you please provide more specific information or clarify your question?


**Whisper as TTS**

In [ ]:
!pip install numpy==1.24.1 pandas==2.0.3 --upgrade


In [ ]:
!pip install openai gtts TTS -q


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.22.0 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 1.22.0 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.

In [ ]:
!pip install nest_asyncio


In [ ]:
import openai
from TTS.api import TTS
import os
import asyncio
from openai import AsyncOpenAI
import nest_asyncio

# Apply nest_asyncio
nest_asyncio.apply()

# Replace 'your-api-key' with your actual OpenAI API key
openai.api_key = '---'

# Initialize the AsyncOpenAI client
client = AsyncOpenAI()

async def generate_text(prompt):
    response = await client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

def text_to_speech(text, filename="output.wav"):
    model_name = "tts_models/en/ljspeech/tacotron2-DDC"
    tts = TTS(model_name=model_name, progress_bar=True, gpu=False)
    tts.tts_to_file(text=text, file_path=filename)
    return filename

async def chat_with_speaking_ai(prompt):
    # Generate response from OpenAI
    response_text = await generate_text(prompt)
    print("Generated Text:", response_text)

    # Convert response text to speech
    audio_file = text_to_speech(response_text)
    print(f"Audio saved as {audio_file}")

    # Play the audio file
    if os.name == 'posix':
        os.system(f"mpg321 {audio_file}")
    elif os.name == 'nt':
        os.system(f"start {audio_file}")

if __name__ == "__main__":
    prompt = "Explain the principles of AI and robotics in simple terms."
    asyncio.run(chat_with_speaking_ai(prompt))


Generated Text: Sure, I'd be happy to explain that in simplistic terms.

1. Artificial Intelligence (AI): This is like teaching a computer or machine how to mimic human intelligence. So, it's not just about following a set of instructions like in traditional programming, it's about understanding, learning, and improving over time. AI involves processes like learning (the acquisition of information and rules for using the information), reasoning (using the rules to reach approximate or definite conclusions) and self-correction.

   AI is generally categorized as either narrow (or weak), where the AI is designed to perform a narrow task like voice recognition, or general (or strong) AI, where the systems can perform any intellectual task that a human being can.

2. Robotics: This is a field overlapping with AI, stressing more on creating physical machines that can carry out tasks. Imagine you have a dog and you teach it tricks - sit, fetch, roll over, etc. Robots are similar, except they

 97%|█████████▋| 109M/113M [00:01<00:00, 65.3MiB/s]

 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--hifigan_v2



100%|██████████| 113M/113M [00:06<00:00, 17.9MiB/s]

 24%|██▎       | 900k/3.80M [00:00<00:00, 8.95MiB/s]
 66%|██████▌   | 2.50M/3.80M [00:00<00:00, 13.1MiB/s]

 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample


100%|██████████| 3.80M/3.80M [00:13<00:00, 13.1MiB/s]

these two can also work together – robotic process automation rpa for instance, is a technology that uses ai for handling repetitive tasks.
 [!] Character '–' not found in the vocabulary. Discarding it.
 > Processing time: 478.18121361732483
 > Real-time factor: 3.34857816144664
Audio saved as output.wav
